In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

## STEP1: LOADING MNIST TRAIN DATASET

In [ ]:
train_dateset=datasets.MNIST('./data/', train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dateset=datasets.MNIST('./data/', train=False,
                            transform=transforms.ToTensor(),
                            download=True)

In [ ]:
print(train_dateset.data.shape, train_dateset.targets.shape)
print(test_dateset.data.shape, test_dateset.targets.shape)

## STEP2: MAKE DATASET ITERABLE

In [ ]:
batch_size=100
num_epochs=5

train_loader=DataLoader(dataset=train_dateset,
                        batch_size=batch_size,
                        shuffle=True)

test_loader=DataLoader(dataset=test_dateset,
                        batch_size=batch_size,
                        shuffle=False)

## STEP3: CREATE MODEL CLASS

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim,
                 layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        self.hidden_dim=hidden_dim
        self.layer_dim=layer_dim
        self.rnn=nn.RNN(input_size=input_dim,
                        hidden_size=hidden_dim,
                        num_layers=layer_dim)